# Keyphrase Extraction using Glove Model 

## 1. Importing Libraries

In [35]:
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import nltk,re, pprint
from nltk.chunk.regexp import *
from nltk.corpus import stopwords
from nltk.chunk.regexp import tag_pattern2re_pattern 


## 2. Text Processing 

In [36]:

# Set Parameters
n_clusters = 100    # No. Of Clusters
stop_words = set(stopwords.words('english'))
Actual_Summary = "output.txt"

def text_processing(file):
    f = open(file, 'r', encoding='utf-8')
    text = f.read()
    
    #Tokenization : Convert to sentences and then to words
    sentences = nltk.sent_tokenize(text)
    
    data = []
    
    for sent in sentences:
        words = nltk.word_tokenize(sent)
        words = [w.lower() for w in words if len(w)>2 and w not in stop_words]
        data.append(words)
        
    return data

lines = text_processing("input.txt")
print("Text Processing : done")


count_frequency = {}
for line in lines:
    for w in line:
        count_frequency[w] = 0
        
for line in lines:
    for w in line:
        count_frequency[w] = count_frequency[w] + 1
            
candidate_term = len(count_frequency.keys())

print(candidate_term)


Text Processing : done
152


## 3. Glove Model Applied

In [61]:
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(lines, window=40)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=5, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [62]:
glove.add_dictionary(corpus.dictionary)
# print(glove.word_vectors[glove.dictionary['bombay']])
print(glove)

## 4. Processing Words 

In [63]:
import numpy as np
import matplotlib.pyplot as plt

#importing word cloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#function to print wordcloud for different words
def cloud(c):
    wordcloud = WordCloud(max_font_size=50, max_words=100,collocations=False).generate_from_frequencies(c)
    plt.figure()
    plt.imshow(wordcloud, interpolation = "bilinear" )
    plt.axis("off")
    plt.show()
    
# Processing Model and Words list 
model = []
words = []
for i in lines:
    for j in i:
        if j not in words:
            words.append(j)
            
for word in words:
    similar=glove.most_similar(word,number=10)
    value = []
    for val in similar:
        value.append(val[1])
    model.append(value)

print("Glove Model : Done")

Glove Model : Done


## 5. Applied Hierarchical Clustering

In [64]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

l = linkage(model, method='complete', metric='seuclidean')

# calculate full dendrogram
plt.figure(figsize=(250, 100))
plt.title('Hierarchical Clustering Dendrogram')
plt.ylabel('word')
plt.xlabel('distance')

dendrogram(
    l,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=16.,  # font size for the x axis labels
    orientation='left',
#     leaf_label_func=lambda v: str(model.index2word[v])
)
# plt.show()
print("Hierarchical Clustering : done")


Hierarchical Clustering : done


## 6. Applied K-Means and Regular Expression Parser

In [81]:
from scipy.spatial import distance
from sklearn.cluster import KMeans

from sklearn.cluster import AgglomerativeClustering
n_clusters=int(candidate_term*4/5)

print("n_clusters : ",n_clusters)
hc = AgglomerativeClustering(n_clusters, affinity="euclidean", linkage='complete')

y_hc = hc.fit_predict(model)

print("***************************************************************************************************")

#Some More Processing

models={}
for i in range(len(words)):
    models[words[i]] = model[i]


summary=[]
# Extracting The Keyword by choosing the mean of cluster
def Extracted_Keywords(clus, clus_name):
    
    kmeans = KMeans(n_clusters=1, random_state=0).fit(clus)
    labels = kmeans.labels_
    mean = kmeans.cluster_centers_.reshape(9,1)
    min_to_centre = []
    
    for i in clus:
        i = np.array(i)
        Y = distance.cdist(i.reshape(9,1), mean, 'euclidean')
        sum_Y = np.sum(Y)
        min_to_centre.append(sum_Y)

    min_value = min(min_to_centre)
    ind_value = np.argmin(min_to_centre)
    
#     print(ind_value)
#     print(clus_name[ind_value])
    summary.append(clus_name[ind_value])

    
for j in range(0,n_clusters):
    clus = []
    clus_name = []
    count = 0
    for i in range(len(y_hc)):
        if(y_hc[i] == j):
            count += 1
            clus.append(models[words[i]])
            clus_name.append(words[i])
    
    Extracted_Keywords(clus, clus_name)

    
    
    
    
    
# grammar = "chunk:{(<JJ>)*(<NN | NNS | NNP>)+} " 
# chunkParser= RegexpParser(grammar)
# tagged = nltk.pos_tag(summary)
# result = chunkParser.parse(tagged)
# for subtree in result.subtrees():
#     print(subtree)
# result.draw()



n_clusters :  121
***************************************************************************************************


In [82]:
import collections
summary_frequency = {}

for word in summary:
    summary_frequency[word] = count_frequency[word]
    
od = sorted(summary_frequency.items(), key=lambda x: x[1])
od = od[::-1]

result = []

for i in range(0,20):
    result.append(od[i][0])

    
    
# Regular Expression Parser
grammar = "chunk:{(<JJ>)*(<NN | NNS | NNP>)+} " 
chunkParser= RegexpParser(grammar)
tagged = nltk.pos_tag(result)
result = chunkParser.parse(tagged)
for subtree in result.subtrees():
    print(subtree)
# result.draw()


(S
  the/DT
  (chunk mpaa/NN bittorrent/NN operators/NNS)
  files/VBP
  (chunk lokitorrent/JJ server/NN host/NN networks/NNS)
  december/VBP
  across/IN
  illegally/RB
  links/VBZ
  (chunk others/NNS site/NN)
  sites/VBZ
  (chunk peer-to-peer/JJ industry/NN)
  (chunk file-sharing/JJ systems/NNS))
(chunk mpaa/NN bittorrent/NN operators/NNS)
(chunk lokitorrent/JJ server/NN host/NN networks/NNS)
(chunk others/NNS site/NN)
(chunk peer-to-peer/JJ industry/NN)
(chunk file-sharing/JJ systems/NNS)


In [83]:
right_lis=[]
def getNodes(parent):
    print(parent.label())
    for node in parent:
      
        if type(node) is nltk.Tree:
            if node.label() == 'chunk':
                print("hii")
                #print(node.leaves())
                right_lis.append(node.leaves())
                #print("Sentence:", " ".join(node.leaves()))  
                
            
            else:
                print(node.label())
        else:
            pass
        print(node)  
        
getNodes(result)
print("done")


S
('the', 'DT')
hii
(chunk mpaa/NN bittorrent/NN operators/NNS)
('files', 'VBP')
hii
(chunk lokitorrent/JJ server/NN host/NN networks/NNS)
('december', 'VBP')
('across', 'IN')
('illegally', 'RB')
('links', 'VBZ')
hii
(chunk others/NNS site/NN)
('sites', 'VBZ')
hii
(chunk peer-to-peer/JJ industry/NN)
hii
(chunk file-sharing/JJ systems/NNS)
done


In [84]:

keyphrase=[]
str1=''
for i in range(len(right_lis)):
    for j in range(len(right_lis[i])):
        for k in range(len(right_lis[i][j])-1):
            if right_lis[i][j][k+1] != 'JJ':
             
                str1=str1+right_lis[i][j][k]
                keyphrase.append(str1)
                str1=''
            else:
            
                str1=str1+right_lis[i][j][k] + ' '
    str1=' '
print(right_lis)
print(keyphrase)


[[('mpaa', 'NN'), ('bittorrent', 'NN'), ('operators', 'NNS')], [('lokitorrent', 'JJ'), ('server', 'NN'), ('host', 'NN'), ('networks', 'NNS')], [('others', 'NNS'), ('site', 'NN')], [('peer-to-peer', 'JJ'), ('industry', 'NN')], [('file-sharing', 'JJ'), ('systems', 'NNS')]]
['mpaa', 'bittorrent', 'operators', ' lokitorrent server', 'host', 'networks', ' others', 'site', ' peer-to-peer industry', ' file-sharing systems']


## 7. Results : Precision, Recall, F-Measure

In [85]:

from rouge import Rouge 

def Calculate_Score():
    str2 = ""
    for i in summary:
        str2 = str2 + i
        str2 = str2 + " "
    
    print("Predicted Summary : \n")
    print(str2)

    print("\n\n")
    f = open(Actual_Summary, 'r', encoding='utf-8')
    text = f.read()
    
    print("Actual Summary : \n")
    print(text)

    rouge = Rouge()
    scores = rouge.get_scores(text, str2)
    print("\n\n")
    
    print("F-Measure : ", scores[0]['rouge-1']['f'])
    print("Precision : ", scores[0]['rouge-1']['p'])
    print("Recall    : ", scores[0]['rouge-1']['r'])
    
Calculate_Score()

Predicted Summary : 

videos could swap getting content 2004 settlement 30,000 torrent records another network run body already phoenix october provided programmes like action using struck website four material copied billions victim soon file-sharers since close market copying system online files hide adding copyright allowed number given share copyrighted stunt they black but suprnova.org stark filed began worried instead targeting ability high-profile every losses you take n't edonkey would round aggressively laws came mpaa dvds forced intended america similar december picture uses trackers systems continents succeeded the operators filing appeared campaign well message direct file-sharing also networks clamping hit host server industry peer-to-peer court notices hitting motion lokitorrent sites site warning others hits 100 single people links infringe nets suits bittorrent click illegally across 



Actual Summary : 

It has filed 100 lawsuits against operators of BitTorrent server